## Import thư viện

In [1]:
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, count, when, to_timestamp, split, regexp_replace, row_number, sum, count_distinct
from functools import reduce
import pyspark.pandas as ps

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Khởi tạo Spark Session

In [2]:
spark = SparkSession.builder \
    .appName("Analysis") \
    .getOrCreate()

## Đọc file data

In [4]:
category_df = spark.read.csv("./data/preprocessed_data.csv", header=True, inferSchema=True)

Py4JJavaError: An error occurred while calling o33.csv.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Traver

In [ ]:
category_df.show()

+-----------+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|    category_id|        publish_time|                tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|Jw1Y-zhQURU|     17.14.11|John Lewis Christ...|          John Lewis|  Howto & Style|2017-11-10T07:38:...|"christmas|""john...| 7224515|  55681|   10247|         9479|https://i.ytimg.c...|        

In [ ]:
def dataframe_info(df):
    print(f"{'-'*40}")
    print(f"DataFrame thông tin:")
    print(f"Số dòng: {df.count()}")
    print(f"Số cột: {len(df.columns)}")
    print(f"{'-'*40}")
    print("Schema:")
    df.printSchema()
    print(f"{'-'*40}")
    print("Số giá trị null trong mỗi cột:")
    null_counts = df.select([
        count(when(col(c).isNull(), c)).alias(c) for c in df.columns
    ])
    null_counts.show()

In [ ]:
dataframe_info(category_df)

----------------------------------------
DataFrame thông tin:
Số dòng: 39540
Số cột: 11
----------------------------------------
Schema:
root
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- description: string (nullable = true)

----------------------------------------
Số giá trị null trong mỗi cột:
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-----------+
|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|description|
+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+-------

## Phân tích

### Số category

In [ ]:
category_df.select('category_id').distinct().show()

+--------------------+
|         category_id|
+--------------------+
|           Education|
|              Gaming|
|       Entertainment|
|     Travel & Events|
|Science & Technology|
|              Sports|
|       Howto & Style|
|    Film & Animation|
|      People & Blogs|
|     News & Politics|
|      Pets & Animals|
|    Autos & Vehicles|
|               Music|
|              Comedy|
+--------------------+



In [ ]:
category_df.select('category_id').distinct().count()

14

### Category nằm top trending nhiều nhất (1vid/ngày = 1 lần)

In [ ]:
category_df.groupBy('category_id').count().orderBy("count", ascending=False).show()

+--------------------+-----+
|         category_id|count|
+--------------------+-----+
|               Music|13754|
|       Entertainment| 9124|
|      People & Blogs| 2926|
|    Film & Animation| 2577|
|       Howto & Style| 1928|
|              Sports| 1907|
|              Comedy| 1828|
|              Gaming| 1788|
|     News & Politics| 1225|
|      Pets & Animals|  534|
|Science & Technology|  518|
|           Education|  457|
|    Autos & Vehicles|  144|
|     Travel & Events|   96|
+--------------------+-----+



#### Chuyển thành pandas-on-Spark DataFrame và plot

In [ ]:
pandas_category_df = category_df.pandas_api()
pandas_category_df['category_id'].value_counts().plot.bar()

C:\Users\solit\AppData\Roaming\Python\Python311\site-packages\pyspark\pandas\base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(


### Category có tổng số lượng views cao nhất (Tổng số view (cao nhất) của các vid thuộc category cụ thể)

#### Tổng số vid

In [ ]:
category_df.select('title').distinct().show()

+--------------------+
|               title|
+--------------------+
|Google Pixelbook:...|
|Fall Out Boy - HO...|
|The Poop In My Pa...|
|I Picked My Girlf...|
|New album Open He...|
|WWE fan interrupt...|
|ASOS UNBOXING HAU...|
|Let's talk... YOU...|
|Parenting Habits ...|
|Charles Manson  -...|
|[OFFICIAL VIDEO] ...|
|Maroon 5 Busks in...|
|BTS interview at ...|
|Vance Joy - Like ...|
|SECRETS REVEALED!...|
|Demi Lovato - Sky...|
|What To Buy HER: ...|
|WE'RE MOVING! | L...|
|Proto-Putty in Li...|
|Little Mix - Glor...|
+--------------------+
only showing top 20 rows



In [ ]:
category_df.select('title').distinct().count()

3363

#### Lọc lấy số view cao nhất của từng vid

In [ ]:
windowSpec = Window.partitionBy("title").orderBy(col("views").desc())
windowSpec

In [ ]:
view_df = category_df.withColumn("view_rank", row_number().over(windowSpec))
view_df.show()

+-------------------+--------------------+-----------------+--------------+-------------------+--------------------+------+-----+--------+-------------+--------------------+---------+
|      trending_date|               title|    channel_title|   category_id|       publish_time|                tags| views|likes|dislikes|comment_count|         description|view_rank|
+-------------------+--------------------+-----------------+--------------+-------------------+--------------------+------+-----+--------+-------------+--------------------+---------+
|2017-11-16 00:00:00|#21 How to go FAS...|       Ben Cathro|        Sports|2017-11-05 20:10:16|[Ben Cathro, sick...| 16074|  689|       8|          142|How flipping rad ...|        1|
|2017-11-15 00:00:00|#21 How to go FAS...|       Ben Cathro|        Sports|2017-11-05 20:10:16|[Ben Cathro, sick...| 15818|  686|       8|          141|How flipping rad ...|        2|
|2017-11-14 00:00:00|#21 How to go FAS...|       Ben Cathro|        Sports|2017-

In [ ]:
view_df = view_df.filter(col('view_rank') == 1)
view_df.show()

+-------------------+------------------------+--------------------+----------------+-------------------+-----------------------+--------+------+--------+-------------+--------------------+---------+
|      trending_date|                   title|       channel_title|     category_id|       publish_time|                   tags|   views| likes|dislikes|comment_count|         description|view_rank|
+-------------------+------------------------+--------------------+----------------+-------------------+-----------------------+--------+------+--------+-------------+--------------------+---------+
|2017-11-16 00:00:00|    #21 How to go FAS...|          Ben Cathro|          Sports|2017-11-05 20:10:16|   [Ben Cathro, sick...|   16074|   689|       8|          142|How flipping rad ...|        1|
|2017-11-22 00:00:00|    #VeteransDay: Tha...|   YouTube Spotlight|   Entertainment|2017-11-10 15:07:13|   [vets, veterans d...|  916104| 27405|    2153|         5292|This #VeteransDay...|        1|
|2017

#### Tính tổng số view theo category

In [ ]:
total_view_df = view_df.groupBy('category_id').agg(sum('views').alias('total_views')).orderBy("total_views", ascending=False)
total_view_df = total_view_df.withColumn("total_views", col("total_views").cast("long"))
total_view_df.show()

+--------------------+-----------+
|         category_id|total_views|
+--------------------+-----------+
|               Music| 6944183379|
|       Entertainment| 1747446715|
|    Film & Animation|  368732367|
|              Comedy|  327819293|
|      People & Blogs|  304066688|
|              Sports|  196526000|
|              Gaming|  115670660|
|     News & Politics|   85441934|
|       Howto & Style|   82587255|
|Science & Technology|   81132989|
|      Pets & Animals|   29851061|
|           Education|   26473589|
|    Autos & Vehicles|   14738146|
|     Travel & Events|   10627822|
+--------------------+-----------+



#### Chuyển thành pandas-on-Spark DataFrame và plot

In [ ]:
pandas_total_view_df = total_view_df.pandas_api()
pandas_total_view_df.plot.bar(x='category_id', y='total_views')

### Category có nhiều kênh youtube làm nhất

In [ ]:
channel_df = category_df.groupBy('category_id').agg(count_distinct('channel_title').alias('total_channels')).orderBy("total_channels", ascending=False)
channel_df.show()

+--------------------+--------------+
|         category_id|total_channels|
+--------------------+--------------+
|               Music|           505|
|       Entertainment|           339|
|      People & Blogs|           188|
|              Gaming|           124|
|    Film & Animation|           103|
|              Sports|            92|
|       Howto & Style|            82|
|              Comedy|            75|
|     News & Politics|            61|
|Science & Technology|            34|
|           Education|            31|
|      Pets & Animals|            25|
|    Autos & Vehicles|            14|
|     Travel & Events|             9|
+--------------------+--------------+



#### Chuyển thành pandas-on-Spark DataFrame và plot

In [ ]:
pandas_channel_df = channel_df.pandas_api()
pandas_channel_df.plot.bar(x='category_id', y='total_channels')